In [1]:
import pandas as pd

In [2]:
# show all columns
pd.set_option('display.max_columns', None)

## Transactions

In [3]:
df_transactions = pd.read_csv('data/raw/bank_s/transactions.csv')

In [4]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88896 entries, 0 to 88895
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tran_id         88896 non-null  int64  
 1   orig_acct       88896 non-null  int64  
 2   bene_acct       88896 non-null  int64  
 3   tx_type         88896 non-null  object 
 4   base_amt        88896 non-null  float64
 5   tran_timestamp  88896 non-null  object 
 6   is_sar          88896 non-null  bool   
 7   alert_id        88896 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(2)
memory usage: 4.8+ MB


In [5]:
df_transactions['tran_timestamp'] = pd.to_datetime(df_transactions['tran_timestamp'])

In [6]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88896 entries, 0 to 88895
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   tran_id         88896 non-null  int64              
 1   orig_acct       88896 non-null  int64              
 2   bene_acct       88896 non-null  int64              
 3   tx_type         88896 non-null  object             
 4   base_amt        88896 non-null  float64            
 5   tran_timestamp  88896 non-null  datetime64[ns, UTC]
 6   is_sar          88896 non-null  bool               
 7   alert_id        88896 non-null  int64              
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), int64(4), object(1)
memory usage: 4.8+ MB


## Accounts

In [7]:
df_accs = pd.read_csv('data/raw/bank_s/accounts.csv')
df_accs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acct_id           100 non-null    int64  
 1   dsply_nm          100 non-null    object 
 2   type              100 non-null    object 
 3   acct_stat         100 non-null    object 
 4   acct_rptng_crncy  100 non-null    object 
 5   prior_sar_count   100 non-null    bool   
 6   branch_id         100 non-null    int64  
 7   open_dt           100 non-null    int64  
 8   close_dt          100 non-null    int64  
 9   initial_deposit   100 non-null    float64
 10  tx_behavior_id    0 non-null      float64
 11  bank_id           100 non-null    object 
 12  first_name        100 non-null    object 
 13  last_name         100 non-null    object 
 14  street_addr       100 non-null    object 
 15  city              100 non-null    object 
 16  state             100 non-null    object 
 17

In [8]:
df_accs['open_dt'].unique()

array([0])

In [9]:
df_accs['close_dt'].unique()

array([1000000])

In [11]:
for col in df_accs.columns:
    # nunique values
    print(f"{col}: {df_accs[col].nunique()} unique values")

acct_id: 100 unique values
dsply_nm: 100 unique values
type: 1 unique values
acct_stat: 1 unique values
acct_rptng_crncy: 1 unique values
prior_sar_count: 2 unique values
branch_id: 1 unique values
open_dt: 1 unique values
close_dt: 1 unique values
initial_deposit: 100 unique values
tx_behavior_id: 0 unique values
bank_id: 1 unique values
first_name: 86 unique values
last_name: 90 unique values
street_addr: 100 unique values
city: 100 unique values
state: 48 unique values
country: 1 unique values
zip: 100 unique values
gender: 2 unique values
birth_date: 100 unique values
ssn: 100 unique values
lon: 100 unique values
lat: 100 unique values


In [10]:
df_accs.columns.tolist()

['acct_id',
 'dsply_nm',
 'type',
 'acct_stat',
 'acct_rptng_crncy',
 'prior_sar_count',
 'branch_id',
 'open_dt',
 'close_dt',
 'initial_deposit',
 'tx_behavior_id',
 'bank_id',
 'first_name',
 'last_name',
 'street_addr',
 'city',
 'state',
 'country',
 'zip',
 'gender',
 'birth_date',
 'ssn',
 'lon',
 'lat']

In [15]:

df_trans = pd.read_csv(f'data/raw/bank_a/transactions.csv')
df_accs = pd.read_csv(f'data/raw/bank_a/accounts.csv')

df_alerts = pd.read_csv(f'data/raw/bank_a/alert_transactions.csv')
df = df_trans.copy()
df['tran_timestamp'] = pd.to_datetime(df['tran_timestamp'], utc=True, errors='coerce')

df['y'] = df['tran_id'].isin(df_alerts['tran_id']).astype('int8')

# select only relevant columns
keep_cols = [
    'acct_id',
    'initial_deposit',
    'state',
    ]
df_accs = df_accs[keep_cols].copy()

df_org = df_accs.rename(columns={c: f"orig_{c}" for c in keep_cols})
df_bene = df_accs.rename(columns={c: f"bene_{c}" for c in keep_cols})

df = df.merge(df_org, how='left', left_on='orig_acct', right_on='orig_acct_id')
df = df.merge(df_bene, how='left', left_on='bene_acct', right_on='bene_acct_id')

df = df.drop(columns=['is_sar', 'alert_id'], errors='ignore')
df = df.drop(columns=['orig_acct_id', 'bene_acct_id'], errors='ignore')
df = df.drop(columns=['tx_type'], errors='ignore')
df = df.sort_values(by='tran_timestamp').reset_index(drop=True)

In [16]:
df.head()

,tran_id,orig_acct,bene_acct,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
0,1,982,23,501.95,2017-01-01 00:00:00+00:00,0,79229.08,AZ,99168.24,WY
1,162,211,740,647.80,2017-01-01 00:00:00+00:00,0,73834.14,CO,84520.52,SC
2,163,863,64,804.72,2017-01-01 00:00:00+00:00,0,89755.39,PA,50082.14,KY
3,164,36,52,692.66,2017-01-01 00:00:00+00:00,0,89970.50,KS,63002.92,UT
4,165,1394,207,590.02,2017-01-01 00:00:00+00:00,0,69667.29,MS,89779.81,VA


In [17]:
df['tran_timestamp'].min(), df['tran_timestamp'].max()

(Timestamp('2017-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2018-12-21 00:00:00+0000', tz='UTC'))

In [18]:
tmp = df.copy()
tmp["month"] = tmp["tran_timestamp"].dt.to_period("M")
print(tmp.groupby("month")["y"].sum())

month
2017-01     3
2017-02     2
2017-03    19
2017-04     2
2017-05     7
2017-06     8
2017-07     4
2017-08     2
2017-09     4
2017-10     3
2017-11     1
2017-12     6
2018-01     1
2018-02     0
2018-03     0
2018-04     1
2018-05     0
2018-06     1
2018-07     1
2018-08     0
2018-09     0
2018-10     0
2018-11     0
2018-12     0
Freq: M, Name: y, dtype: int8


/tmp/ipykernel_2433295/2815097043.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  tmp["month"] = tmp["tran_timestamp"].dt.to_period("M")


In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

df = df[df['tran_timestamp'] <= pd.Timestamp("2018-01-01", tz="UTC")].reset_index(drop=True)

df = df.drop(columns=['orig_acct', 'bene_acct'], errors='ignore')

trainval_test_cutoff = pd.Timestamp("2017-11-01", tz="UTC")
train_val_cutoff = pd.Timestamp("2017-09-01", tz="UTC")

test_mask = df['tran_timestamp'] >= trainval_test_cutoff
val_mask = (df['tran_timestamp'] >= train_val_cutoff) & (df['tran_timestamp'] < trainval_test_cutoff)
train_mask = df['tran_timestamp'] < train_val_cutoff

df_test = df[test_mask].reset_index(drop=True)
df_val = df[val_mask].reset_index(drop=True)
df_train = df[train_mask].reset_index(drop=True)


cat_cols = ["orig_state", "bene_state"]
num_cols = ["base_amt", "orig_initial_deposit", "bene_initial_deposit"]

numeric_pipe = Pipeline([
    ("scaler", StandardScaler()),
    
])

categorical_pipe = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",
)


X_train = df_train[num_cols + cat_cols]
y_train = df_train['y']

X_val = df_val[num_cols + cat_cols]
y_val = df_val['y']

X_test = df_test[num_cols + cat_cols]
y_test = df_test['y']

In [20]:
df_train.tail()

,tran_id,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
55536,55410,483.35,2017-08-31 00:00:00+00:00,0,87264.85,AZ,70798.81,NY
55537,55411,874.10,2017-08-31 00:00:00+00:00,0,75820.34,MD,96021.27,PR
55538,55412,303.81,2017-08-31 00:00:00+00:00,0,63029.65,VA,73519.93,HI
55539,55413,665.54,2017-08-31 00:00:00+00:00,0,71839.01,MA,56058.86,AZ
55540,55391,516.74,2017-08-31 00:00:00+00:00,0,93104.26,WY,98268.42,LA


In [21]:
df_val.tail()

,tran_id,base_amt,tran_timestamp,y,orig_initial_deposit,orig_state,bene_initial_deposit,bene_state
12448,67870,958.39,2017-10-31 00:00:00+00:00,0,86615.79,ME,97715.52,IL
12449,67871,935.79,2017-10-31 00:00:00+00:00,0,60445.79,SD,70030.16,VA
12450,67872,916.73,2017-10-31 00:00:00+00:00,0,74617.96,AZ,97111.38,NJ
12451,67873,944.93,2017-10-31 00:00:00+00:00,0,54748.24,CO,79124.72,AZ
12452,67864,596.23,2017-10-31 00:00:00+00:00,0,82813.09,MS,55712.15,MO


In [22]:
y_val.shape

(12453,)

In [23]:
y_val.sum()

np.int64(7)

In [24]:
y_val.mean()

np.float64(0.0005621135469364812)

In [25]:
y_test.mean()

np.float64(0.0006543890810507619)

In [26]:

X_train_processed = preprocess.fit_transform(X_train)
X_val_processed = preprocess.transform(X_val)